In [39]:
import nltk
import pandas as pd
import numpy as np
import sys
import os

from data_loader import DataLoader

In [87]:
loader = DataLoader(DataLoader.data_path1)
table = loader.load_table()
table.head()

d:\study\NTU_LEARN\AI6122 TEXT MANAGEMENT & PROCESSING\project\data_loader.py:13: FutureWarning: Starting with pandas version 2.0 all arguments of read_json except for the argument 'path_or_buf' will be keyword-only.
  self.table = pd.read_json(data_path, 'records', lines = True);


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AT4EF6GQ3T0Q5,B00JRBN26M,Amazon Customer,"[0, 0]",This was a sweet story. The characters were ...,3,enjoyable read,1398643200,"04 28, 2014"
1,A3RFYXX3H7LRR6,B00HHBV6AS,"Virginia E. Murdy ""MOTHER OF 8""","[0, 0]",Always wonder about angels being among usFaith...,5,a very good series,1388361600,"12 30, 2013"
2,A2CCJVE96Z6JWX,B00A8QWOOQ,swebb,"[0, 0]","The book moved fast, quick interesting read. ...",4,Fast moving captivating,1370995200,"06 12, 2013"
3,A28H2ESL4ZZJEN,B00HZZNBZ4,mandymonkey82,"[4, 4]",Marian Tee has thoughtfully put her paranormal...,5,a wonderful intro,1392595200,"02 17, 2014"
4,ALPNQV67BWS8L,B00FQTN1BA,Thashmira Hemantha,"[0, 0]",Money Flow offers the reader a deadly concocti...,5,money flow,1385164800,"11 23, 2013"


In [4]:
#sentence segmentation
chunk = table['reviewText'][3]

sentences = nltk.sent_tokenize(chunk)
sentences[:3]

['Cat Stevens at his peak with sparse sounds of acoustic guitar and his original band from Tea For The Tillerman.',
 'Good stuff.',
 'Several hits &#34;Moonshadow&#34;, &#34;Peace Train&#34;, &#34;How Can I Tell You&#34;, &#34;If I Laugh&#34;.']

In [5]:
#tokenize
sentence = sentences[0]

tokens = nltk.word_tokenize(sentence)
tokens[:10]

['Cat',
 'Stevens',
 'at',
 'his',
 'peak',
 'with',
 'sparse',
 'sounds',
 'of',
 'acoustic']

In [6]:
#pos tagging
tagged = nltk.pos_tag(tokens)
tagged[:5]

[('Cat', 'NNP'),
 ('Stevens', 'NNP'),
 ('at', 'IN'),
 ('his', 'PRP$'),
 ('peak', 'NN')]

In [95]:
from nltk.corpus import stopwords

def is_word(str):
    for char in str:
        if not char.isalpha():
            return False
    return True 

def calculate_words(dataset):
    word_table = {}
    total_words = 0
    for sentence in dataset:
        tokens = nltk.word_tokenize(sentence)
        for word in tokens:
            if is_word(word):
                total_words += 1
                if word in word_table.keys() and word not in stopwords.words('english'):
                    word_table[word] += 1
                else:
                    word_table[word] = 1

    return total_words, word_table

def get_prob_list(sum, word_table):
    prob_table = {}
    for key, val in word_table.items():
        prob_table[key] = val/sum

    return prob_table

def calculate_relative_entropy(prob_table1, prob_table2):

    RE_list1 = []
    RE_list2 = []
    for key, val in prob_table1.items():

        if key in prob_table2.keys():

            relative_entropy1 = val * np.log(val / prob_table2[key])
            RE_list1.append((key, relative_entropy1))

            relative_entropy2 = prob_table2[key] * np.log(prob_table2[key] / val)
            RE_list2.append((key, relative_entropy2))

    RE_list1.sort(key = lambda x:x[1], reverse = True)
    RE_list2.sort(key = lambda x:x[1], reverse = True)

    return RE_list1, RE_list2

In [96]:
#Measure Indicative words
loader = DataLoader(DataLoader.data_path1)
dataset1 = loader.load_review_text()

loader = DataLoader(DataLoader.data_path2)
dataset2 = loader.load_review_text()

d:\study\NTU_LEARN\AI6122 TEXT MANAGEMENT & PROCESSING\project\data_loader.py:13: FutureWarning: Starting with pandas version 2.0 all arguments of read_json except for the argument 'path_or_buf' will be keyword-only.
  self.table = pd.read_json(data_path, 'records', lines = True);
d:\study\NTU_LEARN\AI6122 TEXT MANAGEMENT & PROCESSING\project\data_loader.py:13: FutureWarning: Starting with pandas version 2.0 all arguments of read_json except for the argument 'path_or_buf' will be keyword-only.
  self.table = pd.read_json(data_path, 'records', lines = True);


In [97]:
sum1, word_table1 = calculate_words(dataset1)
sum2, word_table2 = calculate_words(dataset2)
prob_list1 = get_prob_list(sum1, word_table1)
prob_list2 = get_prob_list(sum2, word_table2)

ind_list1, ind_list2 = calculate_relative_entropy(prob_list1, prob_list2)

In [99]:
ind_list2[:10]

[('song', 0.014766177546912972),
 ('music', 0.0071036697697155135),
 ('The', 0.0055078361602734745),
 ('single', 0.004060209009021712),
 ('On', 0.003848763992664141),
 ('rock', 0.0034322229392648425),
 ('Me', 0.00312575046375914),
 ('best', 0.003022551776981975),
 ('You', 0.002916480127506034),
 ('voice', 0.002754945225920294)]

In [93]:
from nltk.corpus import stopwords

nltk_stopwords = stopwords.words('english')
'on' in nltk_stopwords

True